In [48]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random

class Custom_env(Env):
    def __init__(self):
        # Actions we can take: w+1, r+1, w-1, r-1, w+0
        self.action_space = Discrete(9)
        # Do thay doi min va max.
        # it nhat 1 may va nhieu nhat 4 may
        self.observation_space = 500
        # Set times
        self.times = 1

        # set state
        self.state = 300 + random.randint(-3, 3)

        # tham so dau vao
        self.writeThreshold = 20
        self.readThreshold = 20
        self.V_stalenessThreshold = 5
        
    def step(self, RD, WD, V_staleness):
        # Apply action
        # W: w+1, r+1, w-1, r-1, w+0
        self.state = RD * 10 + WD
        
        # Reduce 1 second
        self.times -= 1 
        
        # Calculate reward
        if WD<self.writeThreshold and RD<self.readThreshold: 
            reward = self.V_stalenessThreshold - V_staleness
        else: 
            reward = -100 
        
        # Check is done
        if self.times <= 0: 
            done = True
        else:
            done = False
        
        # Return step information
        return self.state, reward, done

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        self.times = 60 
        self.state = 300 + random.randint(-3, 3)
        return self.state

In [49]:
env_test = Custom_env()

In [50]:
action_size = env_test.action_space.n
state_size = env_test.observation_space
qtable = np.zeros((state_size, action_size))
print(qtable)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [51]:
qtable.shape

(500, 9)

In [52]:
total_episodes = 10        # Total episodes
learning_rate = 0.8           # Learning rate
max_steps = 9               # Max steps per episode
gamma = 0.95                  # Discounting rate

# Exploration parameters
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.01            # Minimum exploration probability 
decay_rate = 0.001             # Exponential decay rate for exploration prob
#I find that decay_rate=0.001 works much better than 0.01

In [53]:
# List of rewards
rewards = []

# 2 For life or until learning is stopped
for episode in range(total_episodes):
    # Reset the environment
    RD = random.randint(15,20)
    WD = random.randint(15,20)
    state = env_test.reset()
    step = 0
    done = False
    total_rewards = 0
    
    for step in range(max_steps):
        # 3. Choose an action a in the current world state (s)
        ## First we randomize a number
        exp_exp_tradeoff = random.uniform(0, 1)
        
        ## If this number > greater than epsilon --> exploitation (taking the biggest Q value for this state)
        if exp_exp_tradeoff > epsilon:
            action = np.argmax(qtable[state,:])

        # Else doing a random choice --> exploration
        else:
            action= env_test.action_space.sample()

        # Take the action (a) and observe the outcome state(s') and reward (r)
        new_state, reward, done = env_test.step(RD, WD, V_staleness=4)

        # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
        # qtable[new_state,:] : all the actions we can take from new state
        # print(f(action, R, W))
        # R, W = f(action, R, W)
        print(state, action)
        qtable[state, action] = qtable[state, action] + learning_rate * (reward + gamma * np.max(qtable[new_state, :]) - qtable[state, action])
        
        total_rewards =total_rewards + reward
        
        # Our new state is state
        state = new_state
        
        # If done (if we're dead) : finish episode
        if done == True: 
            break
        
    episode += 1
    # Reduce epsilon (because we need less and less exploration)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode) 
    rewards.append(total_rewards)

print ("Score over time: " +  str(sum(rewards)/total_episodes))
print(qtable)
np.save('qtable.npy',np.array(qtable))
print(epsilon)

(301, 4)
(197, 7)
(197, 6)
(197, 4)
(197, 7)
(197, 3)
(197, 6)
(197, 8)
(197, 8)
(298, 3)
(189, 0)
(189, 5)
(189, 3)
(189, 3)
(189, 1)
(189, 1)
(189, 6)
(189, 7)
(301, 7)
(176, 8)
(176, 7)
(176, 2)
(176, 5)
(176, 1)
(176, 3)
(176, 3)
(176, 4)
(301, 3)
(220, 1)
(220, 5)
(220, 4)
(220, 7)
(220, 7)
(220, 6)
(220, 5)
(220, 6)
(303, 7)
(187, 1)
(187, 1)
(187, 4)
(187, 4)
(187, 5)
(187, 7)
(187, 7)
(187, 0)
(300, 1)
(199, 7)
(199, 0)
(199, 3)
(199, 6)
(199, 2)
(199, 8)
(199, 8)
(199, 0)
(300, 1)
(176, 0)
(176, 0)
(176, 4)
(176, 1)
(176, 1)
(176, 5)
(176, 4)
(176, 3)
(297, 5)
(190, 8)
(190, 8)
(190, 8)
(190, 7)
(190, 5)
(190, 5)
(190, 8)
(190, 4)
(297, 3)
(196, 6)
(196, 3)
(196, 6)
(196, 7)
(196, 4)
(196, 2)
(196, 8)
(196, 1)
(299, 5)
(170, 3)
(170, 6)
(170, 5)
(170, 1)
(170, 3)
(170, 1)
(170, 6)
(170, 8)
Score over time: -264
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
0.9901493354

In [59]:
qtable[176,:]

array([4.04562453, 5.36463694, 1.87008   , 5.87472604, 5.78691315,
       5.32137623, 0.        , 1.408     , 0.8       ])

In [55]:
def get_x(x):
    if (x >= 4):
        return 4
    elif (x <= 0):
        return 1
    else:
        return x

In [56]:
def f(x, r, w):
    # print(r, w)
    return {
        0: (get_x(r - 1), get_x(w - 1)),
        1: (get_x(r - 1), get_x(w)),
        2: (get_x(r - 1), get_x(w + 1)),
        3: (get_x(r), get_x(w - 1)),
        4: (get_x(r), get_x(w)),
        5: (get_x(r), get_x(w + 1)),
        6: (get_x(r + 1), get_x(w - 1)),
        7: (get_x(r + 1), get_x(w)),
        8: (get_x(r + 1), get_x(w + 1)),
    }[x]

In [57]:
print(sum(qtable[208, :]) == 0)

True


In [58]:
env_test.reset()
R = 2
W = 2
qtable_new = np.load('qtable.npy')
for episode in range(10):
    RD = random.randint(15,20)
    WD = random.randint(15,20)
    state = env_test.reset()
    step = 0
    done = False

    for step in range(1):
        env_test.render()
        # Take the action (index) that have the maximum expected future reward given that state
        if sum(qtable_new[state, :]) == 0:
            action = random.randint(0, 8)
        else:
            print('ok')
            action = np.argmax(qtable_new[state,:])

        R, W = f(action, R, W)
        print(R, W)
        new_state, reward, done = env_test.step(RD, WD, V_staleness=4)
        
        if done:
            break
        state = new_state

ok
(3, 2)
ok
(4, 2)
(4, 2)
ok
(4, 1)
ok
(4, 1)
ok
(3, 1)
ok
(3, 1)
ok
(2, 1)
ok
(2, 1)
ok
(1, 1)
